<a href="https://colab.research.google.com/github/meliluc/marketing-analytics-customer-insights-project/blob/main/04_BI_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 — Preparación de Tablas para BI (Power BI)

Objetivo:
- Generar tablas **dimensionales** (dim_date, dim_campaign, dim_customer)
- Generar tablas **de hechos** (fact_marketing, fact_churn)
- Guardar CSVs en `data/bi/` listos para importar en Power BI

Grano:
- `fact_marketing`: campaña–día
- `fact_churn`: cliente (Customer_ID sintético)

Relaciones esperadas en Power BI:
- fact_marketing.date_key → dim_date.date_key
- fact_marketing.Campaign_ID → dim_campaign.Campaign_ID
- fact_churn.Customer_ID → dim_customer.Customer_ID

## 1) Setup + carga de datos limpios

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Rutas
BASE = Path("/content/drive/MyDrive/marketing-analytics-customer-insights")
CLEAN_DIR = BASE / "data" / "clean"
BI_DIR = BASE / "bi"
BI_DIR.mkdir(parents=True, exist_ok=True)

# Cargas base
df = pd.read_csv(CLEAN_DIR / "marketing_clean.csv", parse_dates=["Date"])

# Otras
rfm_df = pd.read_csv(CLEAN_DIR / 'rfm_clusters.csv')
rfm_path   = CLEAN_DIR / "rfm_clusters.csv"
churn_path = CLEAN_DIR / "churn_dataset.csv"
churn_df = pd.read_csv(churn_path) if churn_path.exists() else None

# Generamos Customer_IDs con variabilidad realista con valores sintéticos (recordar que son datos artificales)
def generate_customer_ids_rng(n_rows, lam=4, max_freq=20, seed=2024):
    rng = np.random.default_rng(seed)
    n_customers = max(1, int(round(n_rows / lam)))
    freq = rng.poisson(lam, size=n_customers).clip(1, max_freq)

    diff = n_rows - int(freq.sum())
    if diff > 0:
        idx = rng.integers(0, n_customers, size=diff)
        np.add.at(freq, idx, 1)
    elif diff < 0:
        reducibles = np.where(freq > 1)[0]
        idx = rng.choice(reducibles, size=-diff, replace=True)
        np.add.at(freq, idx, -1)
    ids = np.repeat(np.arange(n_customers), freq)
    rng.shuffle(ids)
    return ids
if "Customer_ID" not in df.columns:
    df = df.copy()
    df["Customer_ID"] = generate_customer_ids_rng(len(df), lam=4, max_freq=20, seed=2024)

print("df shape:", df.shape)
print("rfm_df:", None if rfm_df is None else rfm_df.shape)
print("churn_df:", None if churn_df is None else churn_df.shape)

df.head(3)

Mounted at /content/drive
df shape: (200000, 23)
rfm_df: (49994, 5)
churn_df: (20001, 12)


,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROAS,Location,...,Engagement_Score,Customer_Segment,Date,CPA,CTR,CVR,CPC,CPM,ROI_pct,Customer_ID
0,1,innovate industries,email,men 18-24,30 days,google ads,0.04,16174.0,6.29,chicago,...,6,health & wellness,2021-01-01,16174.0,0.263267,0.04,31.964427,8415.192508,5.29,44917
1,2,nexgen systems,email,women 35-44,60 days,google ads,0.12,11566.0,5.61,new york,...,7,fashionistas,2021-01-02,11566.0,0.015419,0.12,99.706897,1537.418583,4.61,13423
2,3,alpha innovations,influencer,men 25-34,30 days,youtube,0.07,10200.0,7.18,los angeles,...,1,outdoor adventurers,2021-01-03,10200.0,0.075864,0.07,17.465753,1325.019486,6.18,29572


## 2) Derivados previos en df (Duration_num)

In [ ]:
# Convertir 'Duration' tipo "30 days" -> 30.0
df["Duration_num"] = (
    df["Duration"]
      .astype(str)
      .str.extract(r"(\d+)")
      .astype(int)
)

df[["Duration","Duration_num"]].head(8)


,Duration,Duration_num
0,30 days,30
1,60 days,60
2,30 days,30
3,60 days,60
4,15 days,15
5,15 days,15
6,60 days,60
7,45 days,45


## 3) dim_date (tabla calendario)

In [ ]:
# Calendario entre min y max de Date
cal = pd.DataFrame({"Date": pd.date_range(df["Date"].min(), df["Date"].max(), freq="D")})
cal["date_key"]  = cal["Date"].dt.strftime("%Y%m%d").astype(int)
cal["year"]      = cal["Date"].dt.year
cal["quarter"]   = cal["Date"].dt.quarter
cal["month"]     = cal["Date"].dt.month
cal["month_name"]= cal["Date"].dt.strftime("%b")
cal["week"]      = cal["Date"].dt.isocalendar().week.astype(int)
cal["dow"]       = cal["Date"].dt.dayofweek
cal["dow_name"]  = cal["Date"].dt.day_name()

cal.to_csv(BI_DIR / "dim_date.csv", index=False)
print("✅ dim_date.csv →", (BI_DIR / "dim_date.csv"))

cal.head()


✅ dim_date.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_date.csv


,Date,date_key,year,quarter,month,month_name,week,dow,dow_name
0,2021-01-01,20210101,2021,1,1,Jan,53,4,Friday
1,2021-01-02,20210102,2021,1,1,Jan,53,5,Saturday
2,2021-01-03,20210103,2021,1,1,Jan,53,6,Sunday
3,2021-01-04,20210104,2021,1,1,Jan,1,0,Monday
4,2021-01-05,20210105,2021,1,1,Jan,1,1,Tuesday


## 4) dim_campaign y algunas normalizaciones

In [ ]:
# Partimos de un agregado único por Campaign_ID
dim_campaign = (
    df.groupby("Campaign_ID", as_index=False, observed=True)
      .agg({
          "Company":"first",
          "Campaign_Type":"first",
          "Target_Audience":"first",
          "Channel_Used":"first",
          "Location":"first",
          "Language":"first",
          "Customer_Segment":"first",
          "Duration":"first",
          "Duration_num":"first"
      })
)

# Normalizo Target_Audience en columnas separadas
# valores actuales: "men 18-24", "women 25-34", "all ages", separaremos género y rango de edad
ta = dim_campaign["Target_Audience"].astype(str).str.lower()

# Género: men / women / all / unknown
dim_campaign["Target_Gender"] = ta.str.extract(r"\b(men|women|all)\b", expand=False).fillna("unknown")

# Rango etario: "18-24", "25-34", etc. o 'all ages' / unknown
dim_campaign["Target_AgeRange"] = ta.str.extract(r"(\d{2}-\d{2}|all ages)", expand=False).fillna("unknown")

# Exportar
dim_campaign.to_csv(BI_DIR / "dim_campaign.csv", index=False)
print("✅ dim_campaign.csv →", (BI_DIR / "dim_campaign.csv"))
dim_campaign.head(10)



✅ dim_campaign.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_campaign.csv


,Campaign_ID,Company,Campaign_Type,Target_Audience,Channel_Used,Location,Language,Customer_Segment,Duration,Duration_num,Target_Gender,Target_AgeRange
0,1,innovate industries,email,men 18-24,google ads,chicago,spanish,health & wellness,30 days,30,men,18-24
1,2,nexgen systems,email,women 35-44,google ads,new york,german,fashionistas,60 days,60,women,35-44
2,3,alpha innovations,influencer,men 25-34,youtube,los angeles,french,outdoor adventurers,30 days,30,men,25-34
3,4,datatech solutions,display,all ages,youtube,miami,mandarin,health & wellness,60 days,60,all,all ages
4,5,nexgen systems,email,men 25-34,youtube,los angeles,mandarin,health & wellness,15 days,15,men,25-34
5,6,datatech solutions,display,all ages,instagram,new york,german,foodies,15 days,15,all,all ages
6,7,nexgen systems,email,women 35-44,website,los angeles,spanish,tech enthusiasts,60 days,60,women,35-44
7,8,datatech solutions,search,men 18-24,google ads,los angeles,mandarin,outdoor adventurers,45 days,45,men,18-24
8,9,alpha innovations,social media,women 35-44,facebook,chicago,german,tech enthusiasts,15 days,15,women,35-44
9,10,techcorp,email,women 35-44,instagram,los angeles,english,tech enthusiasts,15 days,15,women,35-44


## 5) dim_customer (churn + rfm + KPIs de referencia)

In [ ]:
# Base mínima: listado único de Customer_ID
dim_customer = df[["Customer_ID"]].drop_duplicates().copy()

# Añadir churn si existe
if churn_df is not None and {"Customer_ID","churn"} <= set(churn_df.columns):
    dim_customer = dim_customer.merge(
        churn_df[["Customer_ID","churn"]].drop_duplicates(),
        on="Customer_ID", how="left"
    )

# Añadir cluster RFM si existe
if rfm_df is not None and {"Customer_ID","Cluster"} <= set(rfm_df.columns):
    dim_customer = dim_customer.merge(
        rfm_df[["Customer_ID","Cluster"]].drop_duplicates(),
        on="Customer_ID", how="left"
    ).rename(columns={"Cluster":"rfm_cluster"})

# Métricas de referencia por cliente (útil para cortes y KPI cards en BI)
agg_ref = (
    df.groupby("Customer_ID", as_index=False, observed=True)
      .agg({
          "Acquisition_Cost":"sum",
          "Clicks":"sum",
          "Impressions":"sum",
          "CTR":"mean",
          "CVR":"mean",
          "ROAS":"mean"
      })
      .rename(columns={
          "Acquisition_Cost":"spend_total",
          "CTR":"ctr_mean",
          "CVR":"cvr_mean",
          "ROAS":"roas_mean"
      })
)
dim_customer = dim_customer.merge(agg_ref, on="Customer_ID", how="left")

# Exportar
dim_customer.to_csv(BI_DIR / "dim_customer.csv", index=False)
print("✅ dim_customer.csv →", (BI_DIR / "dim_customer.csv"))
dim_customer.head(10)

✅ dim_customer.csv → /content/drive/MyDrive/marketing-analytics-customer-insights/bi/dim_customer.csv


,Customer_ID,churn,rfm_cluster,Recency,Frequency,spend_total,spend_total_df,Clicks,Impressions,ctr_mean,cvr_mean,roas_mean
0,0,0.0,1.0,38.0,5.0,71595.0,75971.0,3117,33394,0.103955,0.0480,4.266000
1,1,1.0,1.0,37.0,4.0,55585.0,19399.0,927,2102,0.441009,0.1500,3.800000
2,2,1.0,1.0,138.0,4.0,50885.0,44285.0,1742,32986,0.055534,0.0625,6.677500
3,3,0.0,3.0,193.0,4.0,59350.0,8183.0,941,7733,0.121686,0.0300,2.140000
4,4,1.0,0.0,67.0,2.0,23710.0,52697.0,2116,28210,0.100726,0.0440,4.986000
5,5,0.0,1.0,104.0,5.0,58231.0,44771.0,2876,20639,0.144605,0.0650,4.597500
6,6,1.0,0.0,54.0,2.0,24248.0,23556.0,1559,20017,0.085881,0.0600,5.203333
7,7,0.0,1.0,89.0,4.0,63686.0,66637.0,2269,24820,0.143796,0.0660,4.806000
8,8,1.0,0.0,82.0,3.0,34873.0,40885.0,2240,37097,0.061000,0.0650,3.832500
9,9,1.0,1.0,22.0,5.0,68392.0,58759.0,3901,19275,0.370352,0.0940,4.470000


## 6) Tablas de hechos

### Fact_marketing (grano: Campaña–Día)

In [ ]:
# Generamos tabla de hechos: fact_marketing
fact_marketing = (
    df.groupby(['Customer_ID', 'Campaign_ID', 'Date', 'Channel_Used'], as_index=False, observed=True)
      .agg({
          'Clicks': 'sum',
          'Impressions': 'sum',
          'Acquisition_Cost': 'sum',
          'CPC': 'mean',
          'CPM': 'mean',
          'CTR': 'mean',
          'CVR': 'mean',
          'ROAS': 'mean'
      })
      .rename(columns={'Acquisition_Cost': 'Spend'})
)

fact_marketing['date_key'] = fact_marketing['Date'].dt.strftime('%Y%m%d').astype(int)

# Exportar la tabla corregida a un nuevo CSV
fact_marketing.to_csv(BI_DIR / 'fact_marketing.csv', index=False)

print('✅ fact_marketing.csv actualizado con Customer_ID')
fact_marketing.head()


✅ fact_marketing.csv actualizado con Customer_ID


,Customer_ID,Campaign_ID,Date,Channel_Used,Clicks,Impressions,Spend,CPC,CPM,CTR,CVR,ROAS,date_key
0,0,13685,2021-06-29,youtube,961,6552,18361.0,19.106139,2802.350427,0.146673,0.07,6.39,20210629
1,0,90241,2021-03-27,instagram,182,9508,13085.0,71.895604,1376.209508,0.019142,0.04,4.23,20210327
2,0,125314,2021-04-29,facebook,964,7139,11524.0,11.954357,1614.231685,0.135033,0.04,2.12,20210429
3,0,135035,2021-12-16,instagram,587,7087,17041.0,29.030664,2404.543530,0.082828,0.06,2.92,20211216
4,0,157343,2021-01-28,email,423,3108,15960.0,37.730496,5135.135135,0.136100,0.03,5.67,20210128


### fact_churn (grano: Cliente)

In [ ]:
if churn_df is not None and {"Customer_ID","churn"} <= set(churn_df.columns):
    fact_churn = churn_df[["Customer_ID","churn"]].drop_duplicates().copy()
    fact_churn.to_csv(BI_DIR / "fact_churn.csv", index=False)
    print("✅ Exportado fact_churn.csv en: ", (BI_DIR / "fact_churn.csv"))
    fact_churn.head()
else:
    print("ℹ️ No se encontró churn_dataset.csv con ['Customer_ID','churn']. Se omite fact_churn.")


✅ Exportado fact_churn.csv en:  /content/drive/MyDrive/marketing-analytics-customer-insights/bi/fact_churn.csv


## 7) Tablas de RFM y Churn juntas

In [ ]:
## Unimos datos de RFM y de abandono en una sola tabla para BI
# 1) Unir las tablas a nivel de cliente
customer_data = rfm_df.merge(churn_df, on='Customer_ID', how='inner')

# 2) Calcular el EngagementProxy (Clicks) y unirlo
engagement_proxy = df.groupby('Customer_ID').agg(
    Clicks=('Clicks', 'sum')
).reset_index()

customer_data = customer_data.merge(engagement_proxy, on='Customer_ID', how='left').fillna(0)

# 3) Normalizar EngagementProxy_norm_inv
min_clicks = customer_data['Clicks'].min()
max_clicks = customer_data['Clicks'].max()
customer_data['EngagementProxy_norm_inv'] = 1 - (customer_data['Clicks'] - min_clicks) / (max_clicks - min_clicks)

# 4) Normalizar Recency (Recency_norm)
min_recency = customer_data['Recency'].min()
max_recency = customer_data['Recency'].max()
customer_data['Recency_norm'] = (customer_data['Recency'] - min_recency) / (max_recency - min_recency)

# 5) Crear la columna Riesgo de Abandono
# Puedes ajustar los umbrales si lo necesitas.
customer_data['Riesgo de Abandono'] = np.where(
    (customer_data['Recency_norm'] > 0.4) &
    (customer_data['EngagementProxy_norm_inv'] > 0.4),
    'Alto',
    'Bajo'
)

# 6) Exportar la tabla final a un solo CSV
BI_DIR = Path("/content/drive/MyDrive/marketing-analytics-customer-insights/bi/")
customer_data.to_csv(BI_DIR / 'dim_customer_final.csv', index=False)
print("✅ dim_customer_final.csv ha sido creado.")


## 8) Comprobaciones de archivos exportados necesarios

In [ ]:
print("— Resumen de archivos BI —")
for name in ["dim_date.csv","dim_campaign.csv","dim_customer.csv","fact_marketing.csv","fact_churn.csv"]:
    p = BI_DIR / name
    print(f"{'✅' if p.exists() else '❌'} {name}", end="")
    if p.exists():
        try:
            tmp = pd.read_csv(p, nrows=5)
            print(f" | filas de muestra: {tmp.shape[0]} | columnas: {tmp.shape[1]}")
        except Exception as e:
            print(f" | (no se pudo leer muestra) {e}")
    else:
        print()

— Resumen de archivos BI —
✅ dim_date.csv | filas de muestra: 5 | columnas: 9
✅ dim_campaign.csv | filas de muestra: 5 | columnas: 12
✅ dim_customer.csv | filas de muestra: 5 | columnas: 12
✅ fact_marketing.csv | filas de muestra: 5 | columnas: 13
✅ fact_churn.csv | filas de muestra: 5 | columnas: 2


## 9) Notas y preparación para tablero Power BI

- Se construyeron tablas en formato **esquema estrella**:
  - Dimensiones: `dim_date`, `dim_campaign`, `dim_customer`.
  - Hechos: `fact_marketing`, `fact_churn`.
- Todas las tablas fueron exportadas como CSV a `data/bi/`.
- Están listas para cargarse en **Power BI** y modelar el dashboard ejecutivo de Marketing Analytics.

- **Archivos importados**: `data/bi/dim_date.csv`, `dim_campaign.csv`, `dim_customer.csv`, `fact_marketing.csv`, `fact_churn.csv` (si existe).
- **Relaciones**:
  - `fact_marketing[date_key]` → `dim_date[date_key]`
  - `fact_marketing[Campaign_ID]` → `dim_campaign[Campaign_ID]`
  - `fact_churn[Customer_ID]` → `dim_customer[Customer_ID]`
- **Slicers útiles**: `year`, `month_name`, `Channel_Used`, `Campaign_Type`, `Target_Gender`, `Target_AgeRange`, `rfm_cluster`, `churn`.
- **Ideas de páginas**:
  1. **Overview**: Spend, ROAS, CPA, CTR, CVR, churn rate.
  2. **Canales/Campañas**: ROAS y CPA por canal/campaña; serie temporal Spend vs ROAS.
  3. **RFM**: % por `rfm_cluster`, barras de Monetary/Recency medios por cluster.
  4. **Churn**: churn rate por canal/cluster; tabla de clientes (si corresponde).

👉 Próximo paso: en Power BI se definirán relaciones entre tablas y se crearán medidas DAX (ej. ROAS promedio, CPA medio, churn rate).
